In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader


train_df = pd.read_csv("training_data.csv")
test_df  = pd.read_csv("test_data.csv")

X_train = train_df[["X1", "X2"]].values.astype("float32")
y_train = train_df["Label"].values.astype("float32").reshape(-1, 1)

X_test = test_df[["X1", "X2"]].values.astype("float32")
y_test = test_df["Label"].values.astype("float32").reshape(-1, 1)


x_mean = X_train.mean(axis=0, keepdims=True)
x_std  = X_train.std(axis=0, keepdims=True) + 1e-8  

X_train_std = (X_train - x_mean) / x_std
X_test_std  = (X_test  - x_mean) / x_std   

X_train_t = torch.tensor(X_train_std, dtype=torch.float32)
y_train_t = torch.tensor(y_train,     dtype=torch.float32)

X_test_t  = torch.tensor(X_test_std,  dtype=torch.float32)
y_test_t  = torch.tensor(y_test,      dtype=torch.float32)

train_ds = TensorDataset(X_train_t, y_train_t)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)


class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 8),
            nn.ReLU(),
            nn.Linear(8, 1) 
        )
    def forward(self, x):
        return self.net(x)

model = SimpleMLP()


criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(
    model.parameters(), 
    lr=0.05, 
    momentum=0.9, 
    weight_decay=1e-4
)


num_epochs = 200

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    
    for xb, yb in train_dl:
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * xb.size(0)
    
    epoch_loss /= len(train_ds)
    
    if (epoch + 1) % 20 == 0:
        model.eval()
        with torch.no_grad():
            train_logits = model(X_train_t)
            train_probs  = torch.sigmoid(train_logits)
            train_preds  = (train_probs >= 0.5).float()
            train_acc    = (train_preds == y_train_t).float().mean().item()
        print(f"Epoch {epoch+1:3d} | loss = {epoch_loss:.4f} | train acc = {train_acc:.4f}")


model.eval()
with torch.no_grad():
    test_logits = model(X_test_t)
    test_probs  = torch.sigmoid(test_logits)
    test_preds  = (test_probs >= 0.5).float()
    
    test_acc = (test_preds == y_test_t).float().mean().item()
    train_logits = model(X_train_t)
    train_probs  = torch.sigmoid(train_logits)
    train_preds  = (train_probs >= 0.5).float()
    train_acc    = (train_preds == y_train_t).float().mean().item()

print(f"Final train accuracy: {train_acc:.4f}")
print(f"Final test accuracy : {test_acc:.4f}")


Epoch  20 | loss = 0.5988 | train acc = 0.6604
Epoch  40 | loss = 0.5904 | train acc = 0.6771
Epoch  60 | loss = 0.5880 | train acc = 0.6750
Epoch  80 | loss = 0.6022 | train acc = 0.6792
Epoch 100 | loss = 0.5853 | train acc = 0.6792
Epoch 120 | loss = 0.5849 | train acc = 0.6771
Epoch 140 | loss = 0.5909 | train acc = 0.6792
Epoch 160 | loss = 0.5889 | train acc = 0.6708
Epoch 180 | loss = 0.5867 | train acc = 0.6792
Epoch 200 | loss = 0.5853 | train acc = 0.6792
Final train accuracy: 0.6792
Final test accuracy : 0.7167
